In [43]:
from datetime import datetime
from dotenv import load_dotenv
import os
import pytz
import requests

import pandas as pd 

df = pd.read_pickle('Nonedf.pkl')

df = df.set_index('headers').T  # Set headers as index (headers) and transpose

print(df)

headers 1. From_Currency Code 2. From_Currency Name 3. To_Currency Code  \
value                     USD  United States Dollar                 AUD   

headers 4. To_Currency Name 5. Exchange Rate    6. Last Refreshed  \
value     Australian Dollar       1.57640000  2025-03-06 01:36:01   

headers 7. Time Zone 8. Bid Price 9. Ask Price  
value            UTC   1.57638000   1.57647000  


In [44]:
#i want to change last refreshed dates to AUS time

df.columns

Index(['1. From_Currency Code', '2. From_Currency Name', '3. To_Currency Code',
       '4. To_Currency Name', '5. Exchange Rate', '6. Last Refreshed',
       '7. Time Zone', '8. Bid Price', '9. Ask Price'],
      dtype='object', name='headers')

In [45]:
from datetime import datetime
from zoneinfo import ZoneInfo

x = pd.to_datetime(df['6. Last Refreshed'][0])

x



Timestamp('2025-03-06 01:36:01')

In [46]:
# Assuming x is a datetime object in UTC

x = x.replace(tzinfo=ZoneInfo('UTC'))

# Convert to Sydney time
australia_time = x.astimezone(ZoneInfo('Australia/Sydney'))

print(australia_time)

2025-03-06 12:36:01+11:00


In [47]:
df = df.reset_index(drop=True)


In [48]:
df['6. Last Refreshed'] = australia_time
df['7. Time Zone']  = 'UTC+10'

In [49]:
df

headers,1. From_Currency Code,2. From_Currency Name,3. To_Currency Code,4. To_Currency Name,5. Exchange Rate,6. Last Refreshed,7. Time Zone,8. Bid Price,9. Ask Price
0,USD,United States Dollar,AUD,Australian Dollar,1.57640000,2025-03-06 12:36:01+11:00,UTC+10,1.57638000,1.57647000
